## Toy notebook for v2 pipeline on KF 1.8.0 manifests
* quick start v2 https://www.kubeflow.org/docs/components/pipelines/v2/pipelines/pipeline-basics/
* migration guide https://www.kubeflow.org/docs/components/pipelines/v2/migration/


The default Kubeflow 1.8.0 Notebooks has the following KF SDKs

```console
kfp                           2.4.0
kfp-pipeline-spec             0.2.2
kfp-server-api                2.0.3
```

In [1]:
import sys

In [2]:
from platform import python_version
print (f"current platform python version: {python_version()}")

current platform python version: 3.11.6


In [3]:
# !{sys.executable} -m pip install --user --upgrade kfp==2.4.0

In [4]:
# !{sys.executable} -m pip uninstall -y kfp-server-api
# !{sys.executable} -m pip install --user --upgrade kfp-server-api==2.0.3
# !{sys.executable} -m pip install --user --upgrade kfp-pipeline-spec==0.2.2

In [5]:
!{sys.executable} -m pip list | grep kfp

kfp                           2.4.0
kfp-pipeline-spec             0.2.2
kfp-server-api                2.0.3


## (Optional) Restart kernel

In [6]:
from kfp.dsl.pipeline_task import PipelineTask

def set_res_limit(task: PipelineTask, mem_req="200Mi", cpu_req="2000m", mem_lim="4000Mi", cpu_lim='4000m') -> PipelineTask:
    """set the resource limit for cpu and memory, no cpu and memory requirement sofar.
    should the limit is set to small, the Task Pod would be stopped by kubernetes with OOMKilled status.
    
    Args:
        task(PipelineTask): the KFP PipelineTask which need to be set the cpu and memory limits
        cpu_limit(str): the str representation of cpu limit e.g. '1' as one cpu time, '0.5' as 1/2 cpu time
        mem_limit(str): the str representation of memory limit e.g. '500M' for 500MB RAM
        
    Return:
        (PipelineTask): the PipelineTask with the desired limitations set
    """
    # return task.set_cpu_limit('1').set_memory_limit('500M')
    return task.set_cpu_request(cpu_req)\
            .set_cpu_limit(cpu_lim)\
            .set_memory_request(mem_req)\
            .set_memory_limit(mem_lim)

In [7]:
from kfp import dsl
from kfp import compiler

@dsl.component(base_image='python:3.10.13-bullseye')
def square(x: float) -> float:
    # need to cast to float otherwise will run into an error
    return float(x ** 2)

## use kubectl -n kubeflow-kindfor logs pythagorean-tr97s-2385800675 to show the error log from console
@dsl.component(base_image='python:3.10.13-bullseye')
def add(x: float, z: float) -> float:
    # for some strange reason, the y param name is changed to "true", if i rename to z, it works
    return float(x + z)

@dsl.component(base_image='python:3.10.13-bullseye')
def square_root(x: float) -> float:
    return float(x ** .5)

# @dsl.pipeline(name='pythagorean-theorem-pipeline',
#               description='Solve for the length of a hypotenuse of a triangle with sides length `a` and `b`.',
#               pipeline_root='gs://my-pipelines-bucket',
#               display_name='Pythagorean pipeline.')
@dsl.pipeline
def pythagorean(a: float, b: float) -> float:
    a_sq_task = square(x=a)
    # if the limit is set to small, will be OOMKilled by kubernetes
    #a_sq_task = set_res_limit(task=a_sq_task, cpu_req='500m', mem_req='200M')
    b_sq_task = square(x=b)
    #b_sq_task = set_res_limit(task=b_sq_task, cpu_req='500m', mem_req='200M')

    sum_task = add(x=a_sq_task.output, z=b_sq_task.output)
    #sum_task = set_res_limit(task=sum_task, cpu_req='1', mem_req='500M')

    result_task = square_root(x=sum_task.output)
    #result_task = set_res_limit(task=result_task, cpu_req='1', mem_req='500M')
    return result_task.output

In [8]:
compiler.Compiler().compile(
    pipeline_func=pythagorean,
    package_path="./pythagorean.yaml"
)

In [9]:
from kfp.client import Client
client = Client()
NAMESPACE = client.get_user_namespace()
print(NAMESPACE)

kubeflow-kindfor


/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [10]:
client.list_experiments(namespace=NAMESPACE)

{'experiments': [{'created_at': datetime.datetime(2023, 11, 16, 13, 19, 44, tzinfo=tzlocal()),
                  'description': None,
                  'display_name': 'demo',
                  'experiment_id': '557b0023-3f89-4df6-bf8e-5091a646aa89',
                  'namespace': 'kubeflow-kindfor',
                  'storage_state': 'AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}

In [11]:
# ENABLE_CACHING = True
ENABLE_CACHING = False
EXPERIMENT_NAME = "demo"

run = client.create_run_from_pipeline_func(
    pipeline_func=pythagorean,
    arguments = {"a": 1, "b": 2},
    run_name="my test v2 run",
    experiment_name = EXPERIMENT_NAME,
    namespace=NAMESPACE,
    enable_caching=ENABLE_CACHING,
)